# Poyang Lake Study Area

In [1]:
# load modules
import os
import numpy as np
import pickle
import warnings
import geopandas as gpd
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects


In [2]:
# region of interest (longitude, latitude in WGS84)
# can also be loaded from a .kml polygon
# kml_polygon = os.path.join(os.getcwd(), 'examples', 'PoyangLakeRiverMouth.kml')
# polygon = SDS_tools.polygon_from_kml(kml_polygon)

aoi_poly = gpd.read_file("PoyangLakeIsland.geojson")
g = [i for i in aoi_poly.geometry]
x,y = g[0].exterior.coords.xy
coords = np.dstack((x,y)).tolist()
polygon = coords 
print(polygon)

# date range
dates = ['1985-01-01', '1985-08-01']

# satellite missions
sat_list = ['L5', 'L7', 'L8','S2']

# name of the site
sitename = 'POYANG'

# filepath where data will be stored
filepath_data = os.path.join(os.getcwd(), 'data')

# put all the inputs into a dictionnary
inputs = {
    'polygon': polygon,
    'dates': dates,
    'sat_list': sat_list,
    'sitename': sitename,
    'filepath': filepath_data
        }


DriverError: PoyangLakeIsland.geojson: No such file or directory

In [3]:
## Check if the path exists or not
path_to_check   = os.path.join(filepath_data, sitename)

if os.path.exists(path_to_check) == False: 
    # before downloading the images, check how many images are available for your inputs
    SDS_download.check_images_available(inputs);

    #%% 2. Retrieve images

    # only uncomment this line if you want Landsat Tier 2 images (not suitable for time-series analysis)
    # inputs['include_T2'] = True

    # retrieve satellite images from GEE    
    metadata = SDS_download.retrieve_images(inputs)
else:
    print("You have downloaded files in the your directory! Let's reuse them!") 
    pass 

NameError: name 'filepath_data' is not defined

In [5]:
metadata = SDS_download.get_metadata(inputs) 

In [10]:
settings = { 
    # Temp flag for dirty hack 
    'dirty_hack_flag': False,
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'output_epsg': 28356,       # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': False,    # if True, shows each shoreline detection to the user for validation
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    'color_style': False,       # if True, saves figure as true color image. If False, saves figure as false color image. 
    # add the inputs defined previously
    'inputs': inputs,
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 450,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 150,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
}

In [11]:
# save jpeg of processed satellite image
SDS_preprocess.save_jpg(metadata, settings)

NameError: name 'setting' is not defined

In [ ]:
%matplotlib qt
settings['reference_shoreline'] = SDS_preprocess.get_reference_sl(metadata, settings)
settings['max_dist_ref'] = 100 # max distance (in meters) allowed from the reference shoreline

In [ ]:
#moment of truth
%matplotlib qt
output = SDS_shoreline.extract_shorelines(metadata, settings)

In [ ]:
fig = plt.figure()
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
plt.legend()
mng = plt.get_current_fig_manager()                                         
mng.window.showMaximized()    
fig.set_size_inches([15.76,  8.52])

In [ ]:
%matplotlib qt
transects = SDS_transects.draw_transects(output, settings)